In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import cv2
import tqdm
import keras
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import img_to_array
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.models import Model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.activations import softmax
from keras.activations import elu
from keras.activations import relu
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.optimizers import SGD

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
plt.hist(train_df['diagnosis'])

In [ ]:
rows = 3
columns = 4
fig = plt.figure(figsize=(4*rows, 5*columns))
for i in range(columns*rows):
        image_path = train_df['id_code']
        image_path = image_path[i]
        image_id = train_df['diagnosis']
        image_id = image_id[i]
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        new_img = cv2.resize(img, (224, 224))
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(new_img)
        plt.tight_layout()

In [ ]:
rows = 3
columns = 4
fig = plt.figure(figsize=(4*rows, 5*columns))
for i in range(columns*rows):
        image_path_test = test_df['id_code']
        image_id_test = image_path_test[i]
        img_test = cv2.imread(f'../input/aptos2019-blindness-detection/test_images/{image_id_test}.png')
        img_test = cv2.cvtColor(img_test, cv2.COLOR_BGR2RGB)
        img_test = cv2.resize(img_test, (224, 224))
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id_test)
        plt.imshow(img_test)
        plt.tight_layout()

In [ ]:
train_df.id_code = train_df.id_code.apply(lambda x: x + ".png")
test_df.id_code = test_df.id_code.apply(lambda x: x + ".png")
train_df['diagnosis'] = train_df['diagnosis'].astype('str')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255, 
                                         validation_split=0.15, 
                                         horizontal_flip=True,
                                         vertical_flip=True, 
                                         rotation_range=40, 
                                         zoom_range=0.2, 
                                         shear_range=0.1,
                                        fill_mode='nearest')
    

In [ ]:
imgPath = f"../input/aptos2019-blindness-detection/train_images/cd54d022e37d.png"
img = load_img(imgPath)
data = img_to_array(img)
samples = np.expand_dims(data, 0)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory="../input/aptos2019-blindness-detection/train_images/",
                                                    x_col="id_code",
                                                    y_col="diagnosis",
                                                    batch_size=12,
                                                    class_mode="categorical",
                                                    target_size=(224, 224),
                                                    subset='training',
                                                    shaffle=True,
                                                    seed=12,
                                                    )

In [ ]:
valid_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    directory="../input/aptos2019-blindness-detection/train_images/",
                                                    x_col="id_code",
                                                    y_col="diagnosis",
                                                    batch_size = 12,
                                                    class_mode="categorical",
                                                    target_size=(224, 224),
                                                    subset='validation',
                                                    shaffle=True,
                                                    seed=12
                                                    )

In [ ]:
def create_resnet(img_dim, CHANNEL, n_class):
    input_tensor = Input(shape=(img_dim, img_dim, CHANNEL))
    base_model = ResNet50(weights = 'imagenet', include_top = False, input_tensor = input_tensor)
    base_model.load_weights('../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', by_name=True)
    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(2048, activation = elu)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(1024, activation = elu)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation = elu)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output_layer = Dense(n_class, activation = 'softmax', name = 'Output_layer')(x)
    model_resnet = Model(input_tensor, output_layer)
    
    return model_resnet

NUM_CLASSES = train_df['diagnosis'].nunique()
IMG_DIM = 224
CHANNEL_SIZE = 3
model_resnet = create_resnet(IMG_DIM, CHANNEL_SIZE, NUM_CLASSES)

In [ ]:
model_resnet.compile(
    optimizer = keras.optimizers.SGD(lr = 1e-3,decay=1e-6, momentum=0.9, nesterov=True), 
    loss = keras.losses.categorical_crossentropy,
    metrics = ['accuracy']
)

In [ ]:
BATCH_SIZE = 12
NUM_EPOCHS = 60
NUB_TRAIN_STEPS = train_generator.n // train_generator.batch_size
NUB_VALID_STEPS = valid_generator.n // valid_generator.batch_size

eraly_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_delta=0.0004, patience=2, factor=0.1, min_lr=1e-6, mode='auto',
                              verbose=1)
NUB_TRAIN_STEPS, NUB_VALID_STEPS
model_resnet.fit_generator(
    generator = train_generator,
    steps_per_epoch = NUB_TRAIN_STEPS,
    validation_data = valid_generator,
    validation_steps = NUB_VALID_STEPS,
    epochs = NUM_EPOCHS,
    callbacks = [eraly_stop, reduce_lr],
    verbose = 2
)